# 1. FPCA
Generate static and dynamic feature

In [1]:
import pandas as pd
import numpy as np
import rpy2.robjects as robjects


robjects.r['source']('/home/sally/DSE-BIRCH/fpca.R')
def fda_data_process(dpath, nharm=3):
    """ FPCA to generate static and dynamic feature.
        Returns:
            C: static feature
            C_hat: dynamic feature
    """
    df = pd.read_csv(dpath, header=None, sep='\t')
    label = df.iloc[:, 0]
    if 0 not in label.unique(): # Adjusting the labels to start from 0
        label = label - 1
        label = label.tolist()
    df = df.iloc[:, 1:].astype(np.float32)
    nr, nc = df.shape
    df_matrix = robjects.r['matrix'](robjects.FloatVector(df.to_numpy().flatten()), nrow=nr, ncol=nc, byrow=True)
    result = robjects.r['fd_decomponent'](df_matrix, nharm)
    C = np.array(result[0])
    W = np.array(result[1])
    varprop = np.array(result[2])

    # Our proposed matrix factorization method
    # W = K@K.T
    K = np.linalg.cholesky(W).T
    C_hat = np.dot(C, K)

    return C, C_hat, label, varprop

nharm = 10
C_train, C_hat_train, Y_train, C_varprop = fda_data_process("/home/sally/DSE-BIRCH/UCRArchive_2018/ECG5000/ECG5000_TRAIN.tsv", nharm=nharm)
C_test, C_hat_test, Y_test, C_hat_varprop = fda_data_process("/home/sally/DSE-BIRCH/UCRArchive_2018/ECG5000/ECG5000_TEST.tsv", nharm=nharm)

R[write to console]: Loading required package: splines

R[write to console]: Loading required package: fds

R[write to console]: Loading required package: rainbow

R[write to console]: Loading required package: MASS

R[write to console]: Loading required package: pcaPP

R[write to console]: Loading required package: RCurl

R[write to console]: Loading required package: deSolve

R[write to console]: 
Attaching package: ‘fda’


R[write to console]: The following object is masked from ‘package:graphics’:

    matplot




# 2. Hierarchical clustering with DSE-BIRCH

In [2]:
from clustering import DSEBirchClustering

# Parameters setting
num_cls = 5 # Number of classes for the dataset
bf = 10
thresh1 = 0.2
thresh2 = 100
_lambda = 1e-05
v = False

# Clustering with DSE-BIRCH
Birch = DSEBirchClustering(C_train, C_hat_train, Y_train, C_test, C_hat_test, Y_test, verbose=v)
result_birch_proposed = Birch.Birch_proposed(branching_factor=bf, n_clusters=num_cls, threshold1=thresh1, threshold2=thresh2, _lambda=_lambda)
print(result_birch_proposed)

{'silhouette_coefficient': 0.3493190966883594, 'DB_index': 1.2571855944557093, 'purity': 0.93, 'RI': 0.8785480230174607, 'NMI': 0.622520546838737}
